## Tutorial showing new "HTML" reports

PyGSTi has transitioned to producing HTML (rather than LaTeX/PDF) reports.  The way to generate such report is largely unchanged, with one important exception.  Previously, the `Results` object had various report-generation methods included within it.  We've found this is too restrictive, as we'd sometimes like to generate a report which utilizes the results from multiple runs of GST (to compare the, for instance).  Thus, the `Results` class is now just a container for a `DataSet` and its related `GateSet`s, `GatestringStructure`s, etc.  All of the report-generation capability is now housed in within separate report functions, which we now demonstrate.

This change allows the reports to include **interactive plots** and **switches**, making it easier to compare different types of analysis or data sets. 


### Get some data
We start by performing GST using `do_long_sequence_gst`, as usual, to create a `Results` object (we could also have just loaded one from file).

In [1]:
import pygsti
from pygsti.construction import std1Q_XYI

In [2]:
gs_target = std1Q_XYI.gs_target
fiducials = std1Q_XYI.fiducials
germs = std1Q_XYI.germs
maxLengths = [1,2,4] #,8,16,32,64]

In [3]:
#Generate some data
gs_datagen = gs_target.depolarize(gate_noise=0.1, spam_noise=0.001)
gs_datagen = gs_datagen.rotate(rotate=0.04)
listOfExperiments = pygsti.construction.make_lsgst_experiment_list(gs_target, fiducials, fiducials, germs, maxLengths)
ds = pygsti.construction.generate_fake_data(gs_datagen, listOfExperiments, nSamples=1000,
                                            sampleError="binomial", seed=1234)

In [4]:
#Run GST
gs_target.set_all_parameterizations("TP") #TP-constrained
results = pygsti.do_long_sequence_gst(ds, gs_target, fiducials, fiducials, germs,
                                      maxLengths, verbosity=0)

### Make the reports
Now that we have `results`, we use the methods within `pygsti.report.factory` to generate reports.  These reports are HTML pages that you can open in your browser or email to your friends.  Setting `auto_open=True` makes the finished report open in your web browser automatically.  

In [5]:
#This report is very similar to what was previously called the
# "full" report - but since it's geared toward only single-qubit
# analyses, it's been renamed.
pygsti.report.create_single_qubit_report(results, "tutorial_files/exampleReport.html",
                                         verbosity=3, auto_open=True, confidenceLevel=95)

AttributeError: module 'pygsti.report' has no attribute 'create_single_qubit_report'

In [6]:
#This report is very similar to the "general" PDF report in previous
# versions of pyGSTi, and is geared toward data from one or more
# qubits or qutrits.
pygsti.report.create_general_report(results, "tutorial_files/exampleGenReport.html",
                                    verbosity=3, auto_open=True)

*** Creating workspace ***
  Adding metadata
*** Generating switchboard tables ***
  Adding topSwitchboard:
  topSwitchboard                                took 1.8e-05 seconds
  
  Adding goSwitchboard1:
  goSwitchboard1                                took 3.8e-05 seconds
  
  Adding goSwitchboard2:
  goSwitchboard2                                took 3.2e-05 seconds
  
  Adding maxLSwitchboard1:
  maxLSwitchboard1                              took 1.9e-05 seconds
  
  Adding targetSpamBriefTable:
Added SpamTable._create to hash-ineffective functions
  targetSpamBriefTable                          took 0.007336 seconds
  
  Adding targetGatesBoxTable:
  targetGatesBoxTable                           took 0.034118 seconds
  
  Adding datasetOverviewTable:
  datasetOverviewTable                          took 0.009783 seconds
  
  Adding bestGatesetSpamParametersTable:
Added SpamParametersTable._create to hash-ineffective functions
  bestGatesetSpamParametersTable                took 0.00

AttributeError: 'NoneType' object has no attribute 'gates'

Next, let's analyze the same data two different ways: with and without the TP-constraint (i.e. whether the gates *must* be trace-preserving) and furthermore gauge optmimize each case using several different SPAM-weights.  In each case we'll call `do_long_sequence_gst` with `gaugeOptParams=False`, so that no gauge optimization is done, and then perform several gauge optimizations separately and add these to the `Results` object via its `add_gaugeoptimized` function.

In [9]:
#Case1: TP-constrained GST
tpTarget = gs_target.copy()
tpTarget.set_all_parameterizations("TP")
results_tp = pygsti.do_long_sequence_gst(ds, tpTarget, fiducials, fiducials, germs,
                                      maxLengths, gaugeOptParams=False, verbosity=0)

#Gauge optimize
est = results_tp.estimates['default']
gsFinal = est.gatesets['final iteration estimate']
gsTarget = est.gatesets['target']
for spamWt in [1e-4,1e-3,1e-2,1e-1,1.0]:
    gs = pygsti.gaugeopt_to_target(gsFinal,gsTarget,{'gates':1, 'spam':spamWt})
    est.add_gaugeoptimized({'itemWeights': {'gates':1, 'spam':spamWt}}, gs, "Spam %g" % spamWt)

In [10]:
#Case2: "Full" GST
fullTarget = gs_target.copy()
fullTarget.set_all_parameterizations("full")
results_full = pygsti.do_long_sequence_gst(ds, fullTarget, fiducials, fiducials, germs,
                                      maxLengths, gaugeOptParams=False, verbosity=0)

#Gauge optimize
est = results_full.estimates['default']
gsFinal = est.gatesets['final iteration estimate']
gsTarget = est.gatesets['target']
for spamWt in [1e-4,1e-3,1e-2,1e-1,1.0]:
    gs = pygsti.gaugeopt_to_target(gsFinal,gsTarget,{'gates':1, 'spam':spamWt})
    est.add_gaugeoptimized({'itemWeights': {'gates':1, 'spam':spamWt}}, gs, "Spam %g" % spamWt)

We'll now call the *same* report factory functions but this time instead of passing a single `Results` object as the first argument we'll pass a *dictionary* of them.  This will result in a report that includes switches to select which case ("TP" or "Full") as well as which gauge optimization to display output quantities for.

In [11]:
pygsti.report.create_single_qubit_report({'TP': results_tp, "Full": results_full},
                                         "tutorial_files/exampleMultiReport.html",verbosity=3,
                                         auto_open=True)

*** Generating tables ***
*** Generating plots ***
*** Merging into template file ***
Output written to tutorial_files/exampleMultiReport.html
Opening tutorial_files/exampleMultiReport.html...


In [12]:
pygsti.report.create_general_report({'TP': results_tp, "Full": results_full},
                                    "tutorial_files/exampleMultiGenReport.html",verbosity=3,
                                    auto_open=True)

*** Generating tables ***
*** Generating plots ***
*** Merging into template file ***
Output written to tutorial_files/exampleMultiGenReport.html
Opening tutorial_files/exampleMultiGenReport.html...


Note that these HTML reports are created quickly but require some time to load within a browser.  This is because the typsetting and figure rendering is done within your browser, rather than up front by `pdflatex`.  Also note that the plots in the new HTML reports are  interactive - a primary advantage of the HTML format.

Other report formats such as **`beamer`-class PDF presentation and Powerpoint presentation have been dropped from pyGSTi**.  These presentation formats were rarely used and moreover we feel that the HTML format is able to provide all of the functionality that was present in these discontinued formats.

### Veteran users
Try setting `brief` to `True` to produce more data-dense and useful versions of the reports.

In [13]:
pygsti.report.create_single_qubit_report(results,"tutorial_files/exampleReportBrief.html", 
                                         brief=True,verbosity=3, auto_open=True)
pygsti.report.create_general_report(results,"tutorial_files/exampleGenReportBrief.html", 
                                    brief=True, verbosity=3, auto_open=True)

*** Generating tables ***
*** Generating plots ***
*** Merging into template file ***
Output written to tutorial_files/exampleReportBrief.html
Opening tutorial_files/exampleReportBrief.html...
*** Generating tables ***
*** Generating plots ***
*** Merging into template file ***
Output written to tutorial_files/exampleGenReportBrief.html
Opening tutorial_files/exampleGenReportBrief.html...


### Sharing with others
The report factory functions may be passed a boolean `connected` argument to specify whether online (`True`) or local (`False`) resources should be used by the generated report HTML files.  If `connected=False` (the default, and so what we used in this notebook), then the HTML files rely on the presence of an `offline` folder being in their directory to operate correctly.  The offline folder is automatically generated for you when you call any of the report factory functions, so you typically don't need to worry about it.  

However, *sharing offline reports* (created with `connected=False`) requires the recipeint has pyGSTi's `offline` folder in the same diretory as any report HTML files.  To facilitate sharing, you can get a zipped copy of the offline folder by calling `create_offline_zip` like this:

In [14]:
pygsti.report.create_offline_zip("tutorial_files/")
  #creates .../tutorial_files/offline.zip

If you use `connected=True`, then there's no need for the offline folder -- but you'll need to have an internet connection to load the HTML file.